In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
import mlflow
import pandas as pd
datapath = "data/evaluations/"


# Download all evaluation data.

In [3]:
# Download source data from s3
from gw2ml.data.s3_sync import download_folder_from_s3



download_folder_from_s3(s3_folder_prefix='datasources/gw2/raw/1763489057', local_folder=f"{datapath}/gw2/")
download_folder_from_s3(s3_folder_prefix='datasources/crypto/raw/1763488125', local_folder=f"{datapath}/crypto/")


Source: ost-s3/datasources/gw2/raw/1763489057
Target: data/evaluations//gw2/

Found 60 files to download

✓ +1 Agony Infusion.csv
✓ Auric Sliver.csv
✓ Bag of Coffee Beans.csv
✓ Bag of Sugar.csv
✓ Black Lion Chest.csv
✓ Bowl of Candy Corn Custard.csv
✓ Choya Spine.csv
✓ Congealed Putrescence.csv
✓ Copper Ore.csv
✓ Cup of Spiced Apple Cider.csv
✓ Elder Wood Log.csv
✓ Eye of Kormir.csv
✓ Flawless Snowflake.csv
✓ Fried Golden Dumpling.csv
✓ Glass of Buttered Spirits.csv
✓ Glob of Ectoplasm.csv
✓ Gold Ingot.csv
✓ Gold Ore.csv
✓ Gossamer Scrap.csv
✓ Green Wood Log.csv
✓ Handful of Red Lentils.csv
✓ Iron Ore.csv
✓ Jug of Water.csv
✓ Jute Scrap.csv
✓ Large Bone.csv
✓ Large Claw.csv
✓ Large Fang.csv
✓ Large Scale.csv
✓ Ley-Infused Sand.csv
✓ Lucent Mote.csv
✓ Lucky Draketail.csv
✓ Lucky Prismatic Rocket.csv
✓ Lump of Coal.csv
✓ Mithril Ore.csv
✓ Mystic Coin.csv
✓ Orichalcum Ore.csv
✓ Piece of Candy Corn.csv
✓ Piece of Common Unidentified Gear.csv
✓ Piece of Unidentified Gear.csv
✓ Piece of Zhai

In [ ]:
from darts import TimeSeries
from sklearn.preprocessing import MinMaxScaler

def prepare_data_gw2_univariate(input, scaling: bool = True, split_percentage=0.8):

    value_cols = ["buy_unit_price"]
    df = pd.read_csv(input, delimiter=";")
    df.head()
    tsdf = df[value_cols + ['fetched_at']].copy()

        # Convert to datetime and set as index
    tsdf['fetched_at'] = pd.to_datetime(tsdf['fetched_at'])
    # Localize timezone to UTC if it has timezone info, or remove it
    if tsdf['fetched_at'].dt.tz is not None:
        tsdf['fetched_at'] = tsdf['fetched_at'].dt.tz_localize(None)
    tsdf = tsdf.set_index('fetched_at')

    # Resample to exact 5-minute intervals, forward-filling missing values
    tsdf_resampled = tsdf[value_cols].resample('5min').mean().interpolate(method='linear')
    series = TimeSeries.from_dataframe(tsdf_resampled, value_cols=value_cols)
    train, test = series.split(split_percentage)

    if scaling:
        scaler = MinMaxScaler()
        train = scaler.fit_transform(train)
        test = scaler.transform(test)

    return train, test


# def prepare_data_crypto():
#
#     return train, test


In [ ]:
import os

# for file in os.listdir(f"{datapath}/crypto/"):
#     print(file)

gw2_serieses =[]
for file in os.listdir(f"{datapath}/gw2/"):
    print(file)
    train, test = prepare_data_gw2_univariate(f"{datapath}/gw2/{file}")

    gw2_serieses.append({"train": train,
                    "test": test,
                    "item": file})


In [ ]:
gw2_serieses